In [3]:
import pandas as pd
import numpy as np
from tsl.datasets import MetrLA
from tsl.ops.imputation import add_missing_values

In [2]:
x = MetrLA()
index, weight = x.get_connectivity()
print(index.shape, weight.shape)

(2, 11753) (11753,)


In [6]:
data, dist, mask = x.load(impute_zeros=False)
data.head()

,773869,767541,767542,717447,717446,717445,773062,767620,737529,717816,...,772167,769372,774204,769806,717590,717592,717595,772168,718141,769373
2012-03-01 00:00:00,64.375000,67.625000,67.125000,61.500000,66.875000,68.750000,65.125,67.125,59.625000,62.750000,...,45.625000,65.500,64.500000,66.428571,66.875,59.375000,69.000000,59.250000,69.000000,61.875
2012-03-01 00:05:00,62.666667,68.555556,65.444444,62.444444,64.444444,68.111111,65.000,65.000,57.444444,63.333333,...,50.666667,69.875,66.666667,58.555556,62.000,61.111111,64.444444,55.888889,68.444444,62.875
2012-03-01 00:10:00,64.000000,63.750000,60.000000,59.000000,66.500000,66.250000,64.500,64.250,63.875000,65.375000,...,44.125000,69.000,56.500000,59.250000,68.125,62.500000,65.625000,61.375000,69.857143,62.000
2012-03-01 00:15:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000
2012-03-01 00:20:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000


In [11]:
data.shape

(34272, 207)

In [8]:
weight

array([1.0000000e+00, 1.0204708e-05, 2.6093593e-01, ..., 6.4949799e-01,
       6.2726212e-07, 1.0000000e+00], dtype=float32)

In [9]:
dist

array([[    0. ,     inf,     inf, ...,     inf,  8114.8, 10009.7],
       [    inf,     0. ,  2504.6, ...,     inf,     inf,     inf],
       [    inf,  1489.3,     0. , ...,     inf,     inf,  9837. ],
       ...,
       [    inf,     inf,     inf, ...,     0. ,     inf,     inf],
       [ 9599.8,     inf,     inf, ...,     inf,     0. ,     inf],
       [10119.9,  9374.8,     inf, ...,     inf,  9018.7,     0. ]],
      dtype=float32)

In [1]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset(lags=14)

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.5)

In [3]:
next(iter(train_dataset))

Data(x=[20, 14], edge_index=[2, 102], edge_attr=[102], y=[20])

In [24]:
for snapshot in train_dataset:
    print(snapshot)
    break

Data(x=[1068, 14], edge_index=[2, 27079], edge_attr=[27079], y=[1068])


In [1]:
import numpy as np
from torch_geometric_temporal.dataset import METRLADatasetLoader
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

loader = METRLADatasetLoader()
dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)

print("Dataset type:  ", dataset)

Dataset type:   <torch_geometric_temporal.signal.static_graph_temporal_signal.StaticGraphTemporalSignal object at 0x7f7d9df57160>


In [7]:
for snapshot in dataset[-2:]:
    print(snapshot.x)

tensor([[[0.6521, 0.6466, 0.7620,  ..., 0.6301, 0.7565, 0.5827],
         [1.4399, 1.4520, 1.4640,  ..., 1.5484, 1.5604, 1.5725]],

        [[0.6356, 0.6246, 0.5455,  ..., 0.6686, 0.5311, 0.6569],
         [1.4399, 1.4520, 1.4640,  ..., 1.5484, 1.5604, 1.5725]],

        [[0.7950, 0.7565, 0.7497,  ..., 0.4267, 0.7785, 0.6569],
         [1.4399, 1.4520, 1.4640,  ..., 1.5484, 1.5604, 1.5725]],

        ...,

        [[0.5806, 0.6356, 0.5765,  ..., 0.6631, 0.6136, 0.6259],
         [1.4399, 1.4520, 1.4640,  ..., 1.5484, 1.5604, 1.5725]],

        [[0.6686, 0.5751, 0.7002,  ..., 0.7249, 0.6576, 0.6692],
         [1.4399, 1.4520, 1.4640,  ..., 1.5484, 1.5604, 1.5725]],

        [[0.5270, 0.4321, 0.3600,  ..., 0.1023, 0.4486, 0.4528],
         [1.4399, 1.4520, 1.4640,  ..., 1.5484, 1.5604, 1.5725]]])
tensor([[[0.6466, 0.7620, 0.7249,  ..., 0.7565, 0.5827, 0.4899],
         [1.4520, 1.4640, 1.4761,  ..., 1.5604, 1.5725, 1.5845]],

        [[0.6246, 0.5455, 0.5270,  ..., 0.5311, 0.6569, 0.6507

In [3]:
loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)

In [4]:
p_fault, p_noise = 0.0015, 0.05
mtrla = add_missing_values(MetrLA(),
                           p_fault=p_fault,
                           p_noise=p_noise,
                           min_seq=12,
                           max_seq=12*4,
                           seed=9101112)

In [5]:
mtrla.dataframe()

nodes,773869,767541,767542,717447,717446,717445,773062,767620,737529,717816,...,772167,769372,774204,769806,717590,717592,717595,772168,718141,769373
channels,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-03-01 00:00:00,64.375000,67.625000,67.125000,61.500000,66.875000,68.750000,65.125000,67.125000,59.625000,62.750000,...,45.625000,65.500000,64.500000,66.428574,66.875000,59.375000,69.000000,59.250000,69.000000,61.875000
2012-03-01 00:05:00,62.666668,68.555557,65.444443,62.444443,64.444443,68.111115,65.000000,65.000000,57.444443,63.333332,...,50.666668,69.875000,66.666664,58.555557,62.000000,61.111111,64.444443,55.888889,68.444443,62.875000
2012-03-01 00:10:00,64.000000,63.750000,60.000000,59.000000,66.500000,66.250000,64.500000,64.250000,63.875000,65.375000,...,44.125000,69.000000,56.500000,59.250000,68.125000,62.500000,65.625000,61.375000,69.857140,62.000000
2012-03-01 00:15:00,64.000000,63.750000,60.000000,59.000000,66.500000,66.250000,64.500000,64.250000,63.875000,65.375000,...,44.125000,69.000000,56.500000,59.250000,68.125000,62.500000,65.625000,61.375000,69.857140,62.000000
2012-03-01 00:20:00,64.000000,63.750000,60.000000,59.000000,66.500000,66.250000,64.500000,64.250000,63.875000,65.375000,...,44.125000,69.000000,56.500000,59.250000,68.125000,62.500000,65.625000,61.375000,69.857140,62.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-06-27 23:35:00,65.000000,65.888885,68.555557,61.666668,32.833332,54.555557,62.444443,63.333332,59.222221,65.333336,...,52.888889,69.000000,65.111115,55.666668,66.333336,62.444443,66.777779,64.888885,69.666664,62.333332
2012-06-27 23:40:00,61.375000,65.625000,66.500000,62.750000,32.833332,50.500000,62.000000,67.000000,65.250000,67.125000,...,54.000000,69.250000,60.125000,60.500000,67.250000,59.375000,66.000000,61.250000,69.000000,62.000000
2012-06-27 23:45:00,67.000000,59.666668,69.555557,61.000000,32.833332,44.777779,64.222221,63.777779,59.777779,57.666668,...,51.333332,67.888885,64.333336,57.000000,66.000000,62.666668,68.666664,63.333332,67.444443,61.222221


In [11]:
mtrla.training_mask.shape

(34272, 207, 1)

In [13]:
mtrla.training_mask.reshape(-1, 207)

array([[ True,  True,  True, ...,  True,  True, False],
       [ True,  True, False, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True, False,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [15]:
mtrla.eval_mask.reshape(-1, 207).shape

(34272, 207)

In [16]:
mtrla.eval_mask[:,:,0].shape

(34272, 207)